# Glacial Cycles (Pollard 1982)
## Amita and Miriam

### Introduction
Ice sheet dynamics is central to understanding Earth's history and future, especially given rising global temperatures. Melting of ice sheets contributes directly to sea level changes which has negative impacts on many communities and wildlife. However, the long term dynamics of ice sheet melt and formation, caused by changes in Earth's orbit, have facinating effects. Understanding the long term dynamics well allows us to distinguish expected behavior from abnormal melting. <br>

##### Milankovich Cycles
source: https://science.nasa.gov/science-research/earth-science/milankovitch-orbital-cycles-and-their-role-in-earths-climate/ <br>
Milankovich cycles are small perterbations in Earth's orbit that change the total level of sunlight the planet recieves. The main parameters are eccentricity, or the shape of the orbit, obliquity, or the angle of tilt of Earth's axis, and precession, the direction Earth's axis of rotation is pointed. These three variations occur at well known intervals which align with past ice ages, confirmed from ice core data. (Pollard 1982). The main cycles occur in intervals of 20, 40, and 100 kyr. Previous models have captures the 20 and 40 kyr cycles but failed to capture the 100 kyr cycle. The 100 kyr cycle is especially prevelant in Northern Hemisphere ice sheets which is the focus of this paper. 

##### Previous Work


### Equations
The thickness, $H$ of an ice sheet over time can be modeled with a nonlinear diffusion equation:
$$ \frac{\partial H}{\partial t} = A \frac{\partial}{\partial x} \left[H^{\alpha}\left| \frac{\partial(H + h)}{\partial x}^{\beta}\right|\frac{\partial(H + h)}{\partial x}\right] + G(H + h, x, \text{orbit})$$ 

$t$: time <br>
$x$: distance to the south <br>
$h$: the elevation of the bedrock surface above a fixed reference level (taken as the present mean sea level) <br>

For this model, the bedrock will always be in isostatic equilibrium with the ice load so $h = -rH$, where r is the ice/rock density ratio. <br>

For this model,<br>
$A = 5.77\times10^{-4} \text{m}^{-3}\text{yr}^{-1}$<br>
$\alpha = 5$<br>
$\beta = 2$<br> 
$r = 0.3$<br>

$G$ is the net annual mass balance on the ice sheet surface, depending on snowfall and ice melt, and is given by<br>

$$ G = \begin{cases}
a(h+h' -E) - b(h+h'-E)^2 & h+h' - E \leq 1500 \text{ m} \\
0.56 & h+h' - E > 1500 \text{ m}
\end{cases} \text{     m yr}^{-1} $$
where $E$ is defined 
$$E = E_0(x) + k \Delta Q$$
$E_0(x)$: the present equlibrium line<br>
$k$: insolation sensitivity <br>
$\Delta Q$: is the difference in the summer half-year insolation at $55 \degree$ N from that of the present <br>

##### Boundary Conditions & Initial Conditons
To find the boundary conditions, we assume the cross-section of the ice sheet is a rectangle with length  $4889 \text{ km}$ (the distance between model boundaries of $30 \degree$ N and $74 \degree$ N). The height is calculated by dividing the given ice sheet cross-sectional area from Pollard by the length of the ice sheet. When $t = 700$ (700,000 years ago), the ice sheet cross sectional area is $4.5 \times 10^9 \text{ m}^2$. Thus the height of the ice sheet is $920.43 \text{ m}$. Repeating the calculations for the other boundary, at $t = 0$ (present day), the cross-sectional area is $3.5 \times 10^9 \text{ m}^2$ so the height is $715.89 \text{ m}$ 

##### Assumptions

* What are the fundamental physical/chemical/etc. processes? 
* Relevant equations and potentially where they come from (i.e. how are they derived)
* Describe all terms, parameters, boundary conditions and initial conditions 
* Describe any assumptions being made

### Numerical methods

* Describe the numerical methods used.
* Why did you choose to use these methods
* Code works as intended

### Results

* What simulations did you run with the model?
* Describe the results.
* How do the results compare with any known solutions, or data?
* How do the results change with different methods, time steps, grid spacing, or model parameters?
* What did the model tell you about the process you were studying?
* How could the model be improved in terms of methods used or processes included?